In [53]:
!%pip install --upgrade langchain
!%pip install --upgrade langchain-openai
!%pip install -qU langchain-core langchain-openai
!%pip install --upgrade openai
!pip install langchainhub

/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control


/bin/bash: line 1: fg: no job control


In [54]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("This lab exercise will use the following values:")
    print("Azure OpenAI Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
    print("Azure AI Search: " + os.getenv("AZURE_AI_SEARCH_SERVICE_NAME"))
else: 
    print("No file .env found")

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("OPENAI_API_VERSION")
azure_openai_completion_deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
azure_openai_embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
azure_ai_search_name = os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")
azure_ai_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
azure_ai_search_index_name = os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
azure_ai_search_api_key = os.getenv("AZURE_AI_SEARCH_API_KEY")







This lab exercise will use the following values:
Azure OpenAI Endpoint: https://oai-mtpchatbot-dev-frcent-001.openai.azure.com/
Azure AI Search: srch-mtpchatbot-dev-frcent-001


In [55]:
from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    openai_api_version = os.getenv("OPENAI_EMBEDDING_API_VERSION"),
    model= os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
)

In [56]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient

from azure.search.documents.models import (
    VectorizedQuery
)

from langchain_community.vectorstores.azuresearch import AzureSearch

from langchain_core.tools import tool


from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from openai import AzureOpenAI
import json


In [57]:
index_name: str = "mtp-chatbot-hybrid-search-plus-weblate-plus-websites"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_AI_SEARCH_API_KEY"),
    index_name=index_name,
    embedding_function=azure_openai_embeddings.embed_query,
)

In [91]:
# Perform a similarity search
results = vector_store.similarity_search(
    query="Wie schließe ich einen TruckLive Vertrag ab?",
    k=3,
    search_type="hybrid",
)
print(results[0].page_content)

1

Schritt 1: 
Registrieren Sie sich für MyTruckPoint. 
Anschließend erscheint eine Schaltfläche zur
Registrierung von TruckLive. 
Um den Prozess zu beginnen, klicken Sie bitte
auf „Weiter“. (     )1

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



2

Schritt 2:
Stimmen Sie dem Daimler Rahmenvertrag
für digitale Dienste zu. (     )
Klicken Sie anschließend auf „Bestätigen
und Weiter“. (     )3

2

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



3

Schritt 3:
Stimmen Sie dem TruckLive Vertrag zu. (     )
Klicken Sie anschließend auf „Bestätigen
und Weiter“. (     )

4

5

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



4

Schritt 4:
Definieren Sie einen Zeitpunkt, ab dem TruckLive
für zukünftige Fahrzeuge aktiviert werden soll. (     )
Wählen Sie danach eine Kontaktperson aus dem
eigenen Unternehmen aus, welche die Meldungen 
zu TruckLive bekommen soll. (     ) 
Abschließend klicken Sie auf „Bestätigen“. (     )

Herzlichen Glückwunsch! 
TruckLive ist nun aktiviert und all Ihre Neufahrzeuge
e

In [1]:
# Bringing it together: implement RAG


client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,  # The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
    api_key=azure_openai_api_key,  # The API key for your Azure OpenAI resource.
    api_version="2023-12-01-preview",  # This version supports function calling
)

tools = [
    {
        "type": "function",
        "function": {
            "name": "query_information",
            "description": "Retrieve information about My TruckPoint, TruckLive and everything related from Azure AI search index",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query string to search for relevant data about My TruckPoint and everything",
                    },
                },
                "required": ["query"],
            },
        },
    }
]


# Ask the question

# function for AI Search query
def query_information(messages):
    query = messages[1]["content"]
    print('AI Search started')
    
    results = list(vector_store.similarity_search(
    query=query,
    k=3,
    search_type="hybrid",
    ))

    return results[0].page_content

def run_conversation(messages, tools):

    # Step 1: send the conversation and available functions to GPT
    response = client.chat.completions.create(
        model=azure_openai_completion_deployment_name,
        messages=messages,
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "query_information"}},
        temperature=0.2,
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wants to call a function
    if tool_calls:
        print("Recommended Function call:")
        print(response_message.tool_calls[0])

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
       
        available_functions = {"query_information": query_information}

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            #print(tool_call.function.arguments)
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            ) 
        second_response = client.chat.completions.create(
            model=azure_openai_completion_deployment_name,
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response.model_dump_json(indent=2)
    else:
        return response
       

system_message =  """   
    If the requested information is not available in the retrieved data, please answer with something like that: 
    I'm sorry, but this question about XY is out of the scope of my knowledge as I am an AI assistent designed to provide information related to My TruckPoint. 
    If you have questions to the usage of My TruckPoint, TruckLive or other related topics, I would be happy to assist you.
    If the user asks for a term that you find SIMILIAR but NOT EQUAL in your source data, ask the user first whether he really means this term. If the user confirms, explain the term. If the user doesn't confirm, tell that you can unfortunately not help on this and ask for a new question.
    
    You are a friendly and professional support chatbot for customers of the application MyTruckpoint. 
    Your task is to answer questions about the use of MyTruckpoint and all associated components.
    Only tell that it is not available in his country.
    Your answer shouldn't be longer then round about 150 words. If the user asks a question in language XY, answer to him in language XY.
"""
messages = [
    {"role": "system", "content": system_message},
    {
        "role": "user",
        "content": "Was ist My TruckPoint?",
    },
]

print(messages[1]["content"])
available_functions = {"query_information": query_information}

result = run_conversation(messages, tools)

print("Final response:")
print(result.model_dump_json(indent=2))

# Build the Prompt and Execute against the Azure OpenAI to get the completion
#chain = LLMChain(llm=model_with_tools, prompt=prompt, verbose=False)
#response = chain.invoke({"original_question": query, "search_results": results})
#print(response)

NameError: name 'AzureOpenAI' is not defined